In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import geocoder
import requests
import arrow
import math
import ratelim
from datetime import datetime
from tqdm import tqdm

In [2]:
df = pd.read_csv("brooklyn_100.csv")

In [3]:
%time df.head()

Wall time: 0 ns


,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6360,55,,A5,28 BAY 10TH STREET,,...,1,-,1,"1,547","1,428",1930,1,A5,"725,000",6/27/2017
1,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6360,157,,A5,36 BAY 10TH STREET,,...,1,-,1,"1,547","1,428",1930,1,A5,"800,000",2/27/2017
2,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6361,20,,A5,47 BAY 10TH STREET,,...,1,-,1,"1,933","1,660",1930,1,A5,-,3/4/2017
3,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6367,42,,S1,1730 86TH STREET,,...,1,1,2,"1,342","1,920",1926,1,S1,"1,200,000",3/31/2017
4,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6373,73,,A1,50 BAY 23RD STREET,,...,1,-,1,"2,417","1,672",1930,1,A1,-,1/8/2018


In [4]:
def geolocate(srs):
    """
    Pandas ufunc which operates on a Pandas series, applying geolocation to tag coordinates from street name.
    Handles a variety of inputs:
    (PERSONS KILLED = 0) -> Does nothing.
    (PERSONS KILLED > 0, LOCATION defined, ON STREET NAME any, OFF STREET NAME any, CROSS STREET NAME any) -> Does nothing.
    (PERSONS KILLED > 0, LOCATION undef, ON STREET NAME def, OFF STREET NAME def, CROSS STREET NAME any) -> Geolocates using ON STREET and OFF STREET.
    (PERSONS KILLED > 0, LOCATION undef, ON STREET NAME undef, OFF STREET NAME undef, CROSS STREET NAME undef) -> Does nothing.*
    (PERSONS KILLED > 0, LOCATION undef, ON STREET NAME undef, OFF STREET NAME undef, CROSS STREET NAME def) -> Does nothing.**

    *  A cleanup step in the after stage might then remove these entries.
    ** This was found to be too unreliable to be worth trying.
    """
    address = srs['ADDRESS']
    #g = geocoder.google('{0}, Brooklyn, NY'.format(address))
    g = geocoder.google('{0}'.format(address))
    
    if g.status == 'ZERO_RESULTS' or g.ok is False:
        # Fail.
        return srs
    # Success!
    #print g.latlng, g.ok
    
    srs['LOCATION'] = g.latlng
    srs['LATITUDE'] = g.latlng[0]
    srs['LONGITUDE'] = g.latlng[1]
    return srs

In [9]:
%time df = df.apply(geolocate,axis = 1)
#brooklyn 100 takes 36 seconds
#11 of them are still NaN

Wall time: 12.8 s


In [10]:
df[df["LOCATION"].isnull()]

,SALE PRICE,ADDRESS,APARTMENT NUMBER,BLOCK,BOROUGH,BUILDING CLASS AT PRESENT,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,COMMERCIAL UNITS,EASE-MENT,...,LONGITUDE,LOT,NEIGHBORHOOD,RESIDENTIAL UNITS,SALE DATE,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE,TOTAL UNITS,YEAR BUILT,ZIP CODE
10,"780,000",286 BAY 10TH STREET,,6460,3,A9,A9,01 ONE FAMILY DWELLINGS,-,,...,NaN,61,BATH BEACH,1,1/9/2018,1,1,1,1920,11228
20,"850,000",291 BAY 13TH STREET,,6461,3,A5,A5,01 ONE FAMILY DWELLINGS,-,,...,NaN,289,BATH BEACH,1,3/3/2017,1,1,1,1940,11214
32,-,39 BAY 10TH STREET,,6361,3,B3,B3,02 TWO FAMILY DWELLINGS,-,,...,NaN,24,BATH BEACH,2,11/2/2017,1,1,2,1930,11228
49,-,8618 18TH AVENUE,,6368,3,S2,S2,02 TWO FAMILY DWELLINGS,1,,...,NaN,38,BATH BEACH,2,11/29/2017,1,1,3,1921,11214
55,10,8687 19TH AVENUE,,6372,3,B1,B1,02 TWO FAMILY DWELLINGS,-,,...,NaN,14,BATH BEACH,2,8/16/2017,1,1,2,1920,11214
56,"1,600,000",8615 19TH AVENUE,,6372,3,B2,B2,02 TWO FAMILY DWELLINGS,-,,...,NaN,44,BATH BEACH,2,4/19/2017,1,1,2,1930,11214
57,-,51 BAY 22ND STREET,,6373,3,B2,B2,02 TWO FAMILY DWELLINGS,-,,...,NaN,27,BATH BEACH,2,6/14/2017,1,1,2,1935,11214
64,"1,380,000",8697 15TH AVENUE,,6394,3,B1,B1,02 TWO FAMILY DWELLINGS,-,,...,NaN,34,BATH BEACH,2,5/4/2017,1,1,2,1940,11228
68,"600,000",1552 BENSON AVENUE,,6396,3,B1,B1,02 TWO FAMILY DWELLINGS,-,,...,NaN,38,BATH BEACH,2,10/25/2017,1,1,2,1930,11228
84,"1,120,000",179 BAY 8TH STREET,,6426,3,B1,B1,02 TWO FAMILY DWELLINGS,-,,...,NaN,36,BATH BEACH,2,11/13/2017,1,1,2,1955,11228


In [119]:
%time df.to_csv("brooklyn_updated.csv")

Wall time: 7 ms
